In [1]:
import numpy as np
import os
import json
from lxml import etree
import cv2
from collections import OrderedDict
from tqdm import tqdm

ModuleNotFoundError: No module named 'numpy'

In [2]:
session = '18_01_2023_session_7'
video = 'DJI_0068'
video_location = f"/fs/ess/PAS2136/Kenya-2023/Zebras/session_data/{session}/drone/"
annotation_location = "/cvat_export/"

In [3]:
# video_path = f"{path}/video.mp4"
video_path = f"{video_location}/videos/{video}.MP4"
# annotation_path = f"{path}/tracks.xml"
#annotation_path = f"{annotation_location}{session}-{video}-annotations.xml"
annotation_path = "/fs/ess/PAS2136/Kenya-2023/Zebras/methods_paper_analysis/annotation_process/cvat_export/18_01_2023_session_7-DJI_0068-annotations.xml"

root = etree.parse(annotation_path).getroot()
annotated = dict()
track2end = {}

for track in root.iterfind("track"):
    track_id = int(track.attrib["id"])

    for box in track.iter("box"):
        frame_id = int(box.attrib["frame"])
        keyframe = int(box.attrib["keyframe"])

        if keyframe == 1:
            track2end[track_id] = frame_id

for track in root.iterfind("track"):
    track_id = int(track.attrib["id"])

    for box in track.iter("box"):
        frame_id = int(box.attrib["frame"])
        keyframe = int(box.attrib["keyframe"])

        if frame_id <= track2end[track_id]:
            if annotated.get(track_id) is None:
                annotated[track_id] = OrderedDict()
                
            scaling_factor = 3

            annotated[track_id][frame_id] = [int(float(box.attrib["xtl"])*scaling_factor),
                                                int(float(box.attrib["ytl"])*scaling_factor),
                                                int(float(box.attrib["xbr"])*scaling_factor),
                                                int(float(box.attrib["ybr"])*scaling_factor), keyframe]

In [5]:
tracks_location = "/fs/ess/PAS2136/Kenya-2023/Zebras/methods_paper_analysis/annotation_process/tracks/"

xml_page = etree.Element("annotations")
etree.SubElement(xml_page, "version").text = "1.1"

for track_id in annotated.keys():
    xml_track = etree.Element("track", id=str(track_id), label="Grevy", source="manual")

    for frame_id in annotated[track_id].keys():
        if frame_id == sorted(annotated[track_id].keys())[-1]:
            outside = "1"
        else:
            outside = "0"

        xml_box = etree.Element("box", frame=str(frame_id), outside=outside, occluded="0",
                                keyframe=str(annotated[track_id][frame_id][4]),
                                xtl=f"{annotated[track_id][frame_id][0]:.2f}",
                                ytl=f"{annotated[track_id][frame_id][1]:.2f}",
                                xbr=f"{annotated[track_id][frame_id][2]:.2f}",
                                ybr=f"{annotated[track_id][frame_id][3]:.2f}", z_order="0")

        xml_track.append(xml_box)

    if len(annotated[track_id].keys()) > 0:
        xml_page.append(xml_track)


# # Parse the original XML file
# original_tree = etree.parse(annotation_path)
# original_root = original_tree.getroot()

# # Extract the 'meta' element
# meta = original_root.find('meta')

# # Append 'meta' to the new XML document
# xml_page.append(meta)

xml_document = etree.ElementTree(xml_page)

xml_document.write(f"{tracks_location}/tracks_.xml", xml_declaration=True, pretty_print=True, encoding="utf-8")

# Extract tracks with KABR tools

In [8]:
os.system(f"python3 /fs/ess/PAS2136/Kenya-2023/Zebras/methods_paper_analysis/annotation_process/kabr-tools/tracks_extractor.py {video_path}/{video}.MP4 {tracks_location}/tracks_.xml")

/fs/ess/PAS2136/Kenya-2023/Zebras/session_data/18_01_2023_session_7/drone//videos/DJI_0068.MP4/DJI_0068.MP4 | /fs/ess/PAS2136/Kenya-2023/Zebras/methods_paper_analysis/annotation_process/tracks//tracks_.xml -> mini-scenes/videos|DJI_0068.MP4|DJI_0068


  0%|          | 0/5806 [00:00<?, ?it/s]


0